In [1]:
# packages
import pandas as pd
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
df = pd.read_csv('./raw_data/ny_2019_acs_block_group_data_zeroed.csv', na_values=".", skiprows=[0])

In [3]:
# Handler the removal of fips column for elastic net regression
df = df.iloc[: , 1:]

In [4]:
df.shape

(15463, 621)

In [5]:
from sklearn.model_selection import train_test_split
scaler = preprocessing.StandardScaler()
y = df['PCT_ACS19_5yr_B08134081']
X = df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081']

In [6]:
import numpy as np
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
df=df.dropna()
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

In [8]:
# https://towardsdatascience.com/feature-selection-in-machine-learning-using-lasso-regression-7809c7c2771a

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',ElasticNet())
])
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )
search.fit(X_train,y_train)


Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.608 total time=   2.0s
[CV 2/5] END .................model__alpha=0.1;, score=-0.482 total time=   2.1s
[CV 3/5] END .................model__alpha=0.1;, score=-0.574 total time=   2.4s
[CV 4/5] END .................model__alpha=0.1;, score=-0.556 total time=   2.5s
[CV 5/5] END .................model__alpha=0.1;, score=-0.551 total time=   2.3s
[CV 1/5] END .................model__alpha=0.2;, score=-1.483 total time=   1.4s
[CV 2/5] END .................model__alpha=0.2;, score=-1.292 total time=   1.4s
[CV 3/5] END .................model__alpha=0.2;, score=-1.433 total time=   1.4s
[CV 4/5] END .................model__alpha=0.2;, score=-1.419 total time=   1.6s
[CV 5/5] END .................model__alpha=0.2;, score=-1.296 total time=   1.5s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-2.551 total time=   1.0s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', ElasticNet())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [10]:
search.best_params_


{'model__alpha': 0.1}

In [11]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [12]:
importance = np.abs(coefficients)

In [13]:
result = np.array(df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081'].columns)[importance > 0]

In [14]:
df_columns = pd.read_csv('./raw_data/2019_data_columns.csv', na_values=".")
df_columns

,name,decode
0,Geo_FIPS,FIPS
1,Geo_NAME,Area Name
2,Geo_QName,Qualifying Name
3,Geo_STUSAB,State Postal Abbreviation
4,Geo_SUMLEV,Summary Level
...,...,...
988,ACS19_5yr_B19013A001s,Std. Error: Median Household Income In The Pas...
989,ACS19_5yr_B19013B001,Median Household Income In The Past 12 Months ...
990,ACS19_5yr_B19013B001s,Std. Error: Median Household Income In The Pas...
991,ACS19_5yr_B19013D001,Median Household Income In The Past 12 Months ...


In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_columns.loc[df_columns['name'].isin(result)]

,name,decode
155,PCT_ACS19_5yr_B01001002,% Total: Male
179,PCT_ACS19_5yr_B01001026,% Total: Female
188,PCT_ACS19_5yr_B01001035,% Total: Female: 25 To 29 Years
189,PCT_ACS19_5yr_B01001036,% Total: Female: 30 To 34 Years
191,PCT_ACS19_5yr_B01001038,% Total: Female: 40 To 44 Years
229,ACS19_5yr_B08134019,Total: Car Truck Or Van: 45 To 59 Minutes
230,ACS19_5yr_B08134020,Total: Car Truck Or Van: 60 Or More Minutes
272,ACS19_5yr_B08134062,Total: Public Transportation (Excluding Taxicab): Less Than 10 Minutes
281,ACS19_5yr_B08134071,Total: Public Transportation (Excluding Taxicab): Bus
288,ACS19_5yr_B08134078,Total: Public Transportation (Excluding Taxicab): Bus: 35 To 44 Minutes
